### Setup

In [1]:
!pip install -U -q -r requirements.txt


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import os
import random
from pathlib import Path
import tiktoken
from rich.markdown import Markdown
import numpy as np
from getpass import getpass
import torch
import csv

In [3]:
os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API Key: ")

In [42]:
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
os.environ["WANDB_PROJECT"] = "llmapps"
os.environ["WANDB_NOTEBOOK_NAME"] = "cicero_large_embedding"

In [8]:
import nltk

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma, FAISS
from langchain.document_loaders import TextLoader
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from transformers import BertTokenizer, BertModel


nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
txt = """
I would like to get your all  thoughts on the bond yield increase this week.  I am not worried about the market downturn but the sudden increase in yields. On 2/16 the 10 year bonds yields increased by almost  9 percent and on 2/19 the yield increased by almost 5 percent.

Key Points from the CNBC Article:

* **The “taper tantrum” in 2013 was a sudden spike in Treasury yields due to market panic after the Federal Reserve announced that it would begin tapering its quantitative easing program.**
* **Major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic.**
* **However, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation.**

The recent rise in bond yields and U.S. inflation expectations has some investors wary that a repeat of the 2013 “taper tantrum” could be on the horizon.

The benchmark U.S. 10-year Treasury note climbed above 1.3% for the first time since February 2020 earlier this week, while the 30-year bond also hit its highest level for a year. Yields move inversely to bond prices.

Yields tend to rise in lockstep with inflation expectations, which have reached their highest levels in a decade in the U.S., powered by increased prospects of a large fiscal stimulus package, progress on vaccine rollouts and pent-up consumer demand.

The “taper tantrum” in 2013 was a sudden spike in Treasury yields due to market panic after the Federal Reserve announced that it would begin tapering its quantitative easing program.

Major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic. The Fed and others have maintained supportive tones in recent policy meetings, vowing to keep financial conditions loose as the global economy looks to emerge from the Covid-19 pandemic.

However, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation.

With central bank support removed, bonds usually fall in price which sends yields higher. This can also spill over into stock markets as higher interest rates means more debt servicing for firms, causing traders to reassess the investing environment.

“The supportive stance from policymakers will likely remain in place until the vaccines have paved a way to some return to normality,” said Shane Balkham, chief investment officer at Beaufort Investment, in a research note this week.

“However, there will be a risk of another ‘taper tantrum’ similar to the one we witnessed in 2013, and this is our main focus for 2021,” Balkham projected, should policymakers begin to unwind this stimulus.

Long-term bond yields in Japan and Europe followed U.S. Treasurys higher toward the end of the week as bondholders shifted their portfolios.

“The fear is that these assets are priced to perfection when the ECB and Fed might eventually taper,” said Sebastien Galy, senior macro strategist at Nordea Asset Management, in a research note entitled “Little taper tantrum.”

“The odds of tapering are helped in the United States by better retail sales after four months of disappointment and the expectation of large issuance from the $1.9 trillion fiscal package.”

Galy suggested the Fed would likely extend the duration on its asset purchases, moderating the upward momentum in inflation.

“Equity markets have reacted negatively to higher yield as it offers an alternative to the dividend yield and a higher discount to long-term cash flows, making them focus more on medium-term growth such as cyclicals” he said. Cyclicals are stocks whose performance tends to align with economic cycles.

Galy expects this process to be more marked in the second half of the year when economic growth picks up, increasing the potential for tapering.

## Tapering in the U.S., but not Europe

Allianz CEO Oliver Bäte told CNBC on Friday that there was a geographical divergence in how the German insurer is thinking about the prospect of interest rate hikes.

“One is Europe, where we continue to have financial repression, where the ECB continues to buy up to the max in order to minimize spreads between the north and the south — the strong balance sheets and the weak ones — and at some point somebody will have to pay the price for that, but in the short term I don’t see any spike in interest rates,” Bäte said, adding that the situation is different stateside.

“Because of the massive programs that have happened, the stimulus that is happening, the dollar being the world’s reserve currency, there is clearly a trend to stoke inflation and it is going to come. Again, I don’t know when and how, but the interest rates have been steepening and they should be steepening further.”

## Rising yields a ‘normal feature’

However, not all analysts are convinced that the rise in bond yields is material for markets. In a note Friday, Barclays Head of European Equity Strategy Emmanuel Cau suggested that rising bond yields were overdue, as they had been lagging the improving macroeconomic outlook for the second half of 2021, and said they were a “normal feature” of economic recovery.

“With the key drivers of inflation pointing up, the prospect of even more fiscal stimulus in the U.S. and pent up demand propelled by high excess savings, it seems right for bond yields to catch-up with other more advanced reflation trades,” Cau said, adding that central banks remain “firmly on hold” given the balance of risks.

He argued that the steepening yield curve is “typical at the early stages of the cycle,” and that so long as vaccine rollouts are successful, growth continues to tick upward and central banks remain cautious, reflationary moves across asset classes look “justified” and equities should be able to withstand higher rates.

“Of course, after the strong move of the last few weeks, equities could mark a pause as many sectors that have rallied with yields look overbought, like commodities and banks,” Cau said.

“But at this stage, we think rising yields are more a confirmation of the equity bull market than a threat, so dips should continue to be bought.
""".strip()

In [10]:
# "uncased" means it doesn't make a difference between "model" and "Model"
model_id = "bert-base-uncased"

bert_tokenizer = BertTokenizer.from_pretrained(model_id)
bert_model = BertModel.from_pretrained(model_id)

In [23]:
text_tokenized = bert_tokenizer.encode(txt, padding=True, truncation=True, return_tensors="pt")
text_tokenized.shape

torch.Size([1, 512])

In [25]:
with torch.inference_mode():
    bert_output = bert_model(text_tokenized)

token_embeddings = bert_output.last_hidden_state
token_embeddings.shape

torch.Size([1, 512, 768])

In [36]:
text_embedding = token_embeddings.mean(dim=1)
print(text_embedding.shape)
text_embedding[:3]

torch.Size([1, 768])


tensor([[-3.4370e-01, -2.6878e-01,  4.5390e-01,  9.1782e-02,  3.2367e-01,
         -1.6420e-01, -1.1108e-01,  5.3217e-01, -1.9182e-01, -2.9239e-02,
          1.9545e-01, -2.2920e-01, -2.6688e-01,  3.6670e-01, -9.1278e-02,
          1.9423e-01,  3.3161e-01, -1.7995e-02, -3.0783e-02,  2.4883e-01,
          2.6420e-01, -1.5116e-01,  2.7700e-01,  7.3954e-01,  3.1077e-01,
         -3.2377e-01, -2.0915e-01, -1.5393e-01, -2.5100e-01, -8.5521e-02,
          4.6696e-01,  2.0274e-01, -2.2786e-01, -8.8018e-02,  4.1402e-01,
          9.6118e-02, -2.6970e-01, -3.3411e-01, -1.8252e-01,  1.3497e-01,
         -6.0556e-01, -4.3499e-01, -1.3769e-01,  2.0354e-01, -4.0769e-01,
         -2.7990e-01,  1.7938e-01, -8.2002e-02,  1.7474e-01, -6.5277e-02,
         -4.2070e-01,  2.4607e-01,  1.2269e-01, -4.0680e-01,  2.1800e-01,
          5.1290e-01, -2.2289e-01, -4.2598e-01, -5.6073e-01, -4.4647e-01,
          2.6121e-01, -1.7192e-01, -5.6021e-02, -5.4155e-01, -1.7537e-01,
          7.5585e-02,  2.0123e-01,  4.

In [41]:
text_decoded = bert_tokenizer.decode(text_tokenized[0], skip_special_tokens=True)
Markdown(text_decoded)

i would like to get your all thoughts on the bond yield increase this week. i am not worried about the market      
downturn but the sudden increase in yields. on 2 / 16 the 10 year bonds yields increased by almost 9 percent and on
2 / 19 the yield increased by almost 5 percent. key points from the cnbc article : * * * the “ taper tantrum ” in  
2013 was a sudden spike in treasury yields due to market panic after the federal reserve announced that it would   
begin tapering its quantitative easing program. * * * * * major central banks around the world have cut interest   
rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy   
throughout the pandemic. * * * * * however, the recent rise in yields suggests that some investors are starting to 
anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation. * * the    
recent rise in bond yields and u. s. inflation expectations has some investors wary that a repeat of the 2013 “    
taper tantrum ” could be on the horizon. the benchmark u. s. 10 - year treasury note climbed above 1. 3 % for the  
first time since february 2020 earlier this week, while the 30 - year bond also hit its highest level for a year.  
yields move inversely to bond prices. yields tend to rise in lockstep with inflation expectations, which have      
reached their highest levels in a decade in the u. s., powered by increased prospects of a large fiscal stimulus   
package, progress on vaccine rollouts and pent - up consumer demand. the “ taper tantrum ” in 2013 was a sudden    
spike in treasury yields due to market panic after the federal reserve announced that it would begin tapering its  
quantitative easing program. major central banks around the world have cut interest rates to historic lows and     
launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic. the 
fed and others have maintained supportive tones in recent policy meetings, vowing to keep financial conditions     
loose as the global economy looks to emerge from the covid - 19 pandemic. however, the recent rise in yields       
suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to          
accommodate a potential rise in inflation. with central bank support removed, bonds usually fall in price which    
sends yields higher. this can also spill over into stock markets as higher interest rates means more debt servicing
for firms, causing traders to reassess the investing environment. “

In [43]:
MODEL_NAME = "text-davinci-003"